In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"   #(xxxx is your specific GPU ID)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Activation, Dense, Flatten, GlobalAveragePooling2D
from MyEarlyStopping import MyEarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.preprocessing import LabelEncoder  
import pandas as pd
import timeit

In [3]:
All_data = ImageDataGenerator(preprocessing_function=preprocess_input)
All_data_generator = All_data.flow_from_directory('/data/home/cou/yuli5/Audio/online_data_adaption/Mel_Audio_folder_digits',
                                                batch_size=1,
                                                target_size=(224,224),
                                                class_mode='categorical',
                                               shuffle=False)

Found 30000 images belonging to 10 classes.


In [4]:
image_names = All_data_generator.filenames
image_no = [i.split("/")[1].split("_")[2].split(".")[0] for i in image_names]
image_no = np.array(list(map(int, image_no)))
ALL_participant_class = [i.split("/")[1].split("_")[1] for i in image_names]
ALL_participant_class = np.array(list(map(int, ALL_participant_class)))
command_class = All_data_generator.classes
All_participant_class = tf.keras.utils.to_categorical(ALL_participant_class-1, num_classes=60)
All_command_class = tf.keras.utils.to_categorical(command_class, num_classes=10)
All_command_uniform = All_command_class*0+1/10

In [5]:
All_Inputs = [next(All_data_generator)[0][0] for _ in range(len(All_data_generator))]
All_Inputs = np.array(All_Inputs)

In [6]:
# image_no<=20 means every word repeat 20 times 
s_number = 5 # ALL_participant_class  the subject index

select_indexs_train = (image_no<=19)&(ALL_participant_class<=s_number)
Train_Inputs = All_Inputs[select_indexs_train]
Train_participant_class = All_participant_class[select_indexs_train][:,0:s_number]
Train_participant_uniform = Train_participant_class*0+1/s_number
Train_command_class = All_command_class[select_indexs_train]
Train_command_uniform = Train_command_class*0+1/10

In [7]:
select_indexs_val = (image_no>29)&(image_no<=39)&(ALL_participant_class<=s_number)
Val_Inputs = All_Inputs[select_indexs_val]
Val_participant_class = All_participant_class[select_indexs_val][:,0:s_number]
Val_participant_uniform = Val_participant_class*0+1/s_number
Val_command_class = All_command_class[select_indexs_val]
Val_command_uniform = Val_command_class*0+1/10

In [8]:
select_indexs_test = (image_no>39)&(image_no<=49)&(ALL_participant_class<=s_number)
Test_Inputs = All_Inputs[select_indexs_test]
Test_participant_class = All_participant_class[select_indexs_test][:,0:s_number]
Test_participant_uniform = Test_participant_class*0+1/s_number
Test_command_class = All_command_class[select_indexs_test]
Test_command_uniform = Test_command_class*0+1/10

In [9]:
Perfomance = pd.read_csv('Performance_0915_Digit_Inputs.csv')

In [10]:
actv_fun_1_1 = "relu" 
actv_fun_1_2 = "sigmoid"
shape_1_1 = 128
shape_1_2 = 256
actv_fun_2_1 = "sigmoid" 
actv_fun_2_2 = "sigmoid"
shape_2_1 = 512
shape_2_2 = 512


In [11]:
high_acc = 0
for run in range(0,10):
    
    # feature extraction layers
    resnet_model = ResNet50(input_shape=(224, 224,3), include_top=False, weights="imagenet")
    for layer in resnet_model.layers:
        layer.trainable = False

    d = resnet_model.output.shape[-1] # dimension of last layer

    ###################### model 1 ###################### 
    layer_1_0 = tf.keras.layers.Dense(d,name="weight_1")(resnet_model.output) #times weight before flatten
    layer_1_1 = tf.keras.layers.Flatten(name='flatten_1')(layer_1_0)

    Dense_1_1 = tf.keras.layers.Dense(shape_1_1, activation=actv_fun_1_1,name='fc1_1')
    layer_1_2 = Dense_1_1(layer_1_1)
    Dense_1_2 = tf.keras.layers.Dense(shape_1_2, activation=actv_fun_1_2,name='fc1_2')
    layer_1_3 = Dense_1_2(layer_1_2)

    Dense_1_3 = tf.keras.layers.Dense(s_number, activation='softmax' ,name='participant_output')
    out_layer_1 = Dense_1_3(layer_1_3)

    ###################### model 2 ###################### 
    layer_2_0 = tf.keras.layers.Dense(d,name="weight_2")(resnet_model.output) #times weight before flatten
    layer_2_1 = tf.keras.layers.Flatten(name='flatten_2')(layer_2_0)

    Dense_2_1 = tf.keras.layers.Dense(shape_2_1, activation=actv_fun_2_1,name='fc2_1')
    layer_2_2  = Dense_2_1(layer_2_1)
    Dense_2_2 = tf.keras.layers.Dense(shape_2_2, activation=actv_fun_2_2,name='fc2_2')
    layer_2_3  = Dense_2_2(layer_2_2)

    Dense_2_3 = tf.keras.layers.Dense(10, activation='softmax',name='command_output')
    out_layer_2 = Dense_2_3(layer_2_3)

    ###################### model 1' ###################### 
    layer_1_2_  = Dense_2_1(layer_1_1)
    layer_1_3_  = Dense_2_2(layer_1_2_)
    out_layer_1_ = Dense_2_3(layer_1_3_)

    ###################### model 1' ###################### 
    layer_2_2_  = Dense_1_1(layer_2_1)
    layer_2_3_  = Dense_1_2(layer_2_2_)
    out_layer_2_ = Dense_1_3(layer_2_3_)

    resnet_model = tf.keras.Model(resnet_model.input, [out_layer_1,out_layer_2,out_layer_1_,out_layer_2_])

    # resnet_model.summary() 

    w_1, w_2, w_1_, w_2_ = 1,1,1,1
    ##################### training ############################
    opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
    callbacks = [MyEarlyStopping(monitor1 = 'val_' + resnet_model.layers[-1].name+'_accuracy',
                                  monitor2 = 'val_' + resnet_model.layers[-2].name+'_accuracy',
                                  patience=5,restore_best_weights=True)]
    resnet_model.compile(optimizer=opt, loss=["categorical_crossentropy","categorical_crossentropy","mse","mse"],
                         loss_weights=[w_1, w_2, w_1_, w_2_], metrics=['accuracy'])
    start = timeit.default_timer()
    history1 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)

    for layer in resnet_model.layers[0:175]:
        layer.trainable = True
    history2 = resnet_model.fit(Train_Inputs, 
                               {resnet_model.layers[-2].name:Train_participant_class, 
                                resnet_model.layers[-1].name:Train_command_class,
                                resnet_model.layers[-1].name+"_1":Train_command_uniform, 
                                resnet_model.layers[-2].name+"_1":Train_participant_uniform}, 
                                validation_data=(Val_Inputs,
                                                 {resnet_model.layers[-2].name:Val_participant_class,
                                                  resnet_model.layers[-1].name:Val_command_class,
                                                  resnet_model.layers[-1].name+"_1":Val_command_uniform,
                                                  resnet_model.layers[-2].name+"_1":Val_participant_uniform}), 
                               callbacks=callbacks,
                               batch_size=64,
                               epochs=10)
   
    stop = timeit.default_timer()
    run_time = stop - start
    
    predictions = resnet_model.predict(Test_Inputs)[0]
    predicted_classes = np.argmax(predictions, axis=1)+1
    participant_class = [i.split("/")[1].split("_")[1] for i in All_data_generator.filenames]
    participant_class = np.array(participant_class)[select_indexs_test]
    participant_class = np.array(list(map(int, participant_class)))
    acc_s = round(sum(x == y for x, y in zip(participant_class, predicted_classes)) / len(participant_class),4)
    
    predictions = resnet_model.predict(Test_Inputs)[1]
    predicted_classes = np.argmax(predictions, axis=1)
    true_classes = np.array([i.split("/")[0] for i in All_data_generator.filenames])
    true_classes = np.array(list(map(int, true_classes)))[select_indexs_test]
    acc_c = round(sum(x == y for x, y in zip(true_classes, predicted_classes)) / len(true_classes),4)
    overall_acc = acc_c + acc_s

    Perfomance = Perfomance.append({'Feature_Extract': "ResNet",'Inputs': "MEL",'Time':run_time,
                                    'Partcp_Acc':acc_s,'Command_Acc':acc_c}, ignore_index=True)
    
    if high_acc < overall_acc:
        resnet_model.save('Digital_data_Initial_group_model_0915_MFCC.h5')
        high_acc = overall_acc
        best_index = run
        
    del resnet_model
    run = run + 1

Epoch 1/10
16/16 [==============================] - 4s 278ms/step - loss: 3.6572 - participant_output_loss: 1.5342 - command_output_loss: 2.1150 - command_output_1_loss: 5.2160e-04 - participant_output_1_loss: 0.0075 - participant_output_accuracy: 0.3700 - command_output_accuracy: 0.4700 - command_output_1_accuracy: 0.0930 - participant_output_1_accuracy: 0.2790 - val_loss: 3.1274 - val_participant_output_loss: 1.2402 - val_command_output_loss: 1.8870 - val_command_output_1_loss: 8.9331e-05 - val_participant_output_1_loss: 1.5648e-04 - val_participant_output_accuracy: 0.7880 - val_command_output_accuracy: 0.8120 - val_command_output_1_accuracy: 0.0060 - val_participant_output_1_accuracy: 0.2680
Epoch 2/10
16/16 [==============================] - 2s 147ms/step - loss: 2.8654 - participant_output_loss: 1.0636 - command_output_loss: 1.8015 - command_output_1_loss: 7.8094e-05 - participant_output_1_loss: 2.2513e-04 - participant_output_accuracy: 0.8540 - command_output_accuracy: 0.8880 - c

Epoch 3/10
16/16 [==============================] - 2s 124ms/step - loss: 0.5707 - participant_output_loss: 0.1157 - command_output_loss: 0.4550 - command_output_1_loss: 2.4932e-05 - participant_output_1_loss: 1.5696e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0430 - participant_output_1_accuracy: 0.1820 - val_loss: 0.6213 - val_participant_output_loss: 0.1718 - val_command_output_loss: 0.4496 - val_command_output_1_loss: 1.9262e-05 - val_participant_output_1_loss: 1.8600e-05 - val_participant_output_accuracy: 0.9900 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.1360 - val_participant_output_1_accuracy: 0.1460
Epoch 4/10
16/16 [==============================] - 3s 160ms/step - loss: 0.5068 - participant_output_loss: 0.1041 - command_output_loss: 0.4026 - command_output_1_loss: 3.0726e-05 - participant_output_1_loss: 1.4667e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000

Epoch 5/10
16/16 [==============================] - 3s 190ms/step - loss: 2.0595 - participant_output_loss: 0.4351 - command_output_loss: 1.6243 - command_output_1_loss: 5.0849e-05 - participant_output_1_loss: 2.6163e-05 - participant_output_accuracy: 0.9900 - command_output_accuracy: 0.9950 - command_output_1_accuracy: 0.0440 - participant_output_1_accuracy: 0.2320 - val_loss: 2.0147 - val_participant_output_loss: 0.4408 - val_command_output_loss: 1.5739 - val_command_output_1_loss: 4.7128e-05 - val_participant_output_1_loss: 2.6170e-05 - val_participant_output_accuracy: 0.9780 - val_command_output_accuracy: 0.9880 - val_command_output_1_accuracy: 0.0020 - val_participant_output_1_accuracy: 0.2660
Epoch 6/10
16/16 [==============================] - 3s 157ms/step - loss: 1.8744 - participant_output_loss: 0.3532 - command_output_loss: 1.5210 - command_output_1_loss: 7.4627e-05 - participant_output_1_loss: 2.4153e-05 - participant_output_accuracy: 0.9960 - command_output_accuracy: 0.9990

Epoch 7/10
16/16 [==============================] - 2s 142ms/step - loss: 0.7228 - participant_output_loss: 0.0897 - command_output_loss: 0.6331 - command_output_1_loss: 2.2889e-05 - participant_output_1_loss: 1.1492e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0030 - participant_output_1_accuracy: 0.2060 - val_loss: 0.7629 - val_participant_output_loss: 0.1389 - val_command_output_loss: 0.6240 - val_command_output_1_loss: 2.5402e-05 - val_participant_output_1_loss: 1.2670e-05 - val_participant_output_accuracy: 0.9900 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2160
Epoch 8/10
16/16 [==============================] - 3s 176ms/step - loss: 0.6612 - participant_output_loss: 0.0841 - command_output_loss: 0.5771 - command_output_1_loss: 4.2266e-05 - participant_output_1_loss: 1.0899e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 1.

Epoch 1/10
16/16 [==============================] - 3s 189ms/step - loss: 0.9817 - participant_output_loss: 0.1068 - command_output_loss: 0.8748 - command_output_1_loss: 7.5561e-05 - participant_output_1_loss: 1.7098e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0450 - participant_output_1_accuracy: 0.2090 - val_loss: 1.0102 - val_participant_output_loss: 0.1577 - val_command_output_loss: 0.8524 - val_command_output_1_loss: 5.6080e-05 - val_participant_output_1_loss: 1.8931e-05 - val_participant_output_accuracy: 0.9940 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2480
Epoch 2/10
16/16 [==============================] - 2s 128ms/step - loss: 0.8854 - participant_output_loss: 0.0924 - command_output_loss: 0.7929 - command_output_1_loss: 5.2375e-05 - participant_output_1_loss: 1.5846e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.

...

Epoch 8/10
16/16 [==============================] - 2s 123ms/step - loss: 0.5121 - participant_output_loss: 0.1410 - command_output_loss: 0.3711 - command_output_1_loss: 1.5664e-05 - participant_output_1_loss: 1.3329e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.0130 - participant_output_1_accuracy: 0.2330 - val_loss: 0.5715 - val_participant_output_loss: 0.1962 - val_command_output_loss: 0.3752 - val_command_output_1_loss: 1.4391e-05 - val_participant_output_1_loss: 1.7360e-05 - val_participant_output_accuracy: 0.9940 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.3380
Epoch 9/10
16/16 [==============================] - 2s 118ms/step - loss: 0.4698 - participant_output_loss: 0.1323 - command_output_loss: 0.3375 - command_output_1_loss: 1.7610e-05 - participant_output_1_loss: 1.2176e-05 - participant_output_accuracy: 1.0000 - command_output_accuracy: 0.

Epoch 10/10
16/16 [==============================] - 2s 123ms/step - loss: 1.3697 - participant_output_loss: 0.2063 - command_output_loss: 1.1634 - command_output_1_loss: 7.4208e-05 - participant_output_1_loss: 1.1232e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.0710 - participant_output_1_accuracy: 0.0800 - val_loss: 1.3948 - val_participant_output_loss: 0.2624 - val_command_output_loss: 1.1324 - val_command_output_1_loss: 4.6511e-05 - val_participant_output_1_loss: 1.0409e-05 - val_participant_output_accuracy: 0.9840 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0220 - val_participant_output_1_accuracy: 0.0680
Epoch 1/10
16/16 [==============================] - 3s 171ms/step - loss: 1.2612 - participant_output_loss: 0.1799 - command_output_loss: 1.0812 - command_output_1_loss: 9.7833e-05 - participant_output_1_loss: 1.0802e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.000

Epoch 2/10
16/16 [==============================] - 2s 134ms/step - loss: 3.1041 - participant_output_loss: 1.0315 - command_output_loss: 2.0723 - command_output_1_loss: 2.0850e-04 - participant_output_1_loss: 7.4412e-05 - participant_output_accuracy: 0.8540 - command_output_accuracy: 0.5370 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.1220 - val_loss: 2.9032 - val_participant_output_loss: 0.8877 - val_command_output_loss: 2.0153 - val_command_output_1_loss: 1.1129e-04 - val_participant_output_1_loss: 5.8369e-05 - val_participant_output_accuracy: 0.8860 - val_command_output_accuracy: 0.7460 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.3660
Epoch 3/10
16/16 [==============================] - 2s 139ms/step - loss: 2.7330 - participant_output_loss: 0.7526 - command_output_loss: 1.9802 - command_output_1_loss: 1.2172e-04 - participant_output_1_loss: 4.2968e-05 - participant_output_accuracy: 0.9500 - command_output_accuracy

16/16 [==============================] - 2s 124ms/step - loss: 1.3363 - participant_output_loss: 0.1543 - command_output_loss: 1.1819 - command_output_1_loss: 3.6083e-05 - participant_output_1_loss: 1.3546e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - command_output_1_accuracy: 0.0000e+00 - participant_output_1_accuracy: 0.2240 - val_loss: 1.3670 - val_participant_output_loss: 0.2033 - val_command_output_loss: 1.1637 - val_command_output_1_loss: 1.2413e-05 - val_participant_output_1_loss: 1.5051e-05 - val_participant_output_accuracy: 0.9940 - val_command_output_accuracy: 0.9960 - val_command_output_1_accuracy: 0.0000e+00 - val_participant_output_1_accuracy: 0.2000
Epoch 4/10
16/16 [==============================] - 2s 120ms/step - loss: 1.2553 - participant_output_loss: 0.1402 - command_output_loss: 1.1151 - command_output_1_loss: 1.6846e-05 - participant_output_1_loss: 1.2239e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 1.0000 - 

Epoch 8/10
16/16 [==============================] - 2s 130ms/step - loss: 1.7101 - participant_output_loss: 0.2710 - command_output_loss: 1.4391 - command_output_1_loss: 2.5863e-05 - participant_output_1_loss: 1.4950e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 0.9990 - command_output_1_accuracy: 0.1370 - participant_output_1_accuracy: 0.2290 - val_loss: 1.7187 - val_participant_output_loss: 0.3158 - val_command_output_loss: 1.4029 - val_command_output_1_loss: 1.8377e-05 - val_participant_output_1_loss: 1.5475e-05 - val_participant_output_accuracy: 0.9820 - val_command_output_accuracy: 0.9940 - val_command_output_1_accuracy: 0.3400 - val_participant_output_1_accuracy: 0.2240
Epoch 9/10
16/16 [==============================] - 2s 133ms/step - loss: 1.5853 - participant_output_loss: 0.2350 - command_output_loss: 1.3503 - command_output_1_loss: 4.0806e-05 - participant_output_1_loss: 1.3853e-05 - participant_output_accuracy: 0.9990 - command_output_accuracy: 0.9990

In [12]:
Perfomance

,Command_Acc,Feature_Extract,Inputs,Partcp_Acc,Time
0,0.982,ResNet,SPEC,0.970,53.745848
1,0.980,ResNet,SPEC,0.968,59.788192
2,0.972,ResNet,SPEC,0.960,60.039808
3,0.982,ResNet,SPEC,0.958,56.502710
4,0.980,ResNet,SPEC,0.968,56.194534
5,0.986,ResNet,SPEC,0.966,55.374021
6,0.978,ResNet,SPEC,0.952,55.079770
7,0.988,ResNet,SPEC,0.960,55.301056
8,0.984,ResNet,SPEC,0.960,51.448608
9,0.980,ResNet,SPEC,0.952,52.307214


In [13]:
Perfomance.to_csv('Performance_0915_Digit_Inputs.csv',index=False)